# (Alternative Approach) TimeSeries Exploratory Data Analysis
This notebook takes a different approach to simulate the slider effect. It yields similar (albeit slightly less relevant) results than the main notebook.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error


pd.set_option('display.max_columns', None)

In [2]:
df_base = pd.read_csv("..\Data_Sets\processed\economicData_1995-2022.csv")

In [3]:
#years_delta = [1,6,15,25]
years_delta = [1,2,3,4,5,6,8,10,12,15,20,25]

### TimeSeriesGrowthAnalysis()
To assist us in this journey, let's create the function 'TimeSeriesGrowthAnalysis'. A summary of it:

**Purpose:** This function computes how changes in a predictor variable (like 'Overall Score') relate to changes in a target variable (like 'GDP per capita') over various time ranges.

**Working:**
  - Cleans data and computes an Exponential Moving Average (EMA) for the predictor.
  - For each time period (delta), it:
	- Shifts the data by that period.
	- Calculates the percentage change for the target.
	- Fits a polynomial regression model using the difference from EMA as the predictor.

**Output:** The function outputs regression results (coefficients, intercepts, R^2 scores and RMSE) for each time shift and an augmented DataFrame (df_withDeltas) that contains computed changes for each time period.

In [4]:
def TimeSeriesDoubleDeltaGrowthAnalysis(
        df,
        predictor = 'Overall Score',
        target = 'GDP per capita (current USD)',
        quantiledTarget = 'Country Quintile',
        timeField = 'Index Year',
        mergeFields = ['Country Name', 'Index Year'],
        timePeriods_delta = [1,6,10],
        polyOrder = 1,
        EMAdepth = 3
    ):
    '''
    In a nutshell, this function analyzes time-series data using a time-slider-type analysis.
    
    predictor = the (single) variable name you want to analyze. This function doesn't analyze more than one predictor at once.
    target = the dependent variable name we want to predict for.
    quantiledTarget = categorical data to split the data (useful for later visualization)
    timeField = the name of the time column you're using
    mergeFields = a list to define the Primary/Composite Key of the DataSet.
    timePeriods_delta = list of delta ranges to test the data against
    '''
    
    # Initializing Base DataFrame
    cleaned_data = df[[
                        mergeFields[0],
                        mergeFields[1],
                        predictor,
                        target,
                        quantiledTarget
                    ]].dropna()
    
    # Initializing Lists and DF to return
    coefficients = []
    intercepts = []
    r2_scores = []
    RMSE = []    
    
    # Precompute the EMA and EMA diff for the predictor  
    cleaned_data[f'{predictor}_EMA_{EMAdepth}'] \
        = cleaned_data.groupby('Country Name')[predictor]\
            .transform(lambda x: x.ewm(span=EMAdepth, adjust=False).mean())   
    
    cleaned_data[f'{predictor}_diff_from_EMA'] = cleaned_data[predictor] - cleaned_data[f'{predictor}_EMA_{EMAdepth}']
    
    # Avoid Zero values on diff (Certain regressions break with 0 values here)
    # Further on that, that is because certain fields, like Judicial Effectiveness, have late beginnings.
    cleaned_data[f'{predictor}_diff_from_EMA']\
        = cleaned_data[f'{predictor}_diff_from_EMA'].transform(lambda x: np.random.normal(0, 0.000001) if x == 0 else x)
    
    
    df_withDeltas = cleaned_data.copy() #just initializing, for later appending
    
    # Loop over each year delta, compute & store the merged data, and fit a Regression Model
    for delta in timePeriods_delta:
        
        # ----------------- Time Slider Effect Calculation -----------------
        # Create a shifted dataframe to compute the changes
        shifted_data = cleaned_data[[*mergeFields, target]].copy()
        shifted_data[timeField] -= delta
        
        merged_data = pd.merge(
            cleaned_data,
            shifted_data[[*mergeFields, target]],  # Only include the shifted target
            on=mergeFields,
            suffixes=('', f'_plus_{delta}'),
            indicator=False,
            how='inner'
        )
        
        # If there's no data for this range, go to the next iteration
        if merged_data.shape[0] == 0:
            print(f"No overlapping data for {predictor} with a time delta of {delta}. Appending NaN values for this delta.")

            coefficients.append(np.nan)
            intercepts.append(np.nan)
            r2_scores.append(np.nan)
            RMSE.append(np.nan)
            continue
        
        # Compute the percentage change for target
        merged_data[f'{target}_change_{delta}']\
            = ((merged_data[f'{target}_plus_{delta}']\
               - merged_data[target])\
               / merged_data[target]) * 100
        
        
        # Merge results to df_withDeltas based on mergeFields
        df_withDeltas = pd.merge(
            df_withDeltas,
            merged_data[[*mergeFields, f'{target}_change_{delta}']],
            on=mergeFields,
            how='left'
        )        
        
        # -------------------- Model Building & Fitting --------------------
        X = merged_data[f'{predictor}_diff_from_EMA'].values.reshape(-1, 1)
        y = merged_data[f'{target}_change_{delta}']

        # Generate polynomial features    
        poly = PolynomialFeatures(degree=polyOrder)
        
        X_poly = poly.fit_transform(X)
        
        
        # Instantiate Regression
        model = LinearRegression().fit(X_poly, y)

        # Predict using the polynomial features
        yHat = model.predict(X_poly)

        # Store the results
        coefficients.append(sum(np.abs(model.coef_)))  # Note: This will now represent only the maximum term of the polynomial
        intercepts.append(model.intercept_)
        r2_scores.append(model.score(X_poly, y))
        RMSE.append(np.sqrt(mean_squared_error(y, yHat)))

    # Create a results DataFrame
    regression_results = pd.DataFrame({
        'Years Ahead': timePeriods_delta,
        'Coefficient (target to predictor % Change)': coefficients,
        'Intercept': intercepts,
        'R^2 Score': r2_scores,
        'RMSE': RMSE
    })
    
    return regression_results, df_withDeltas


#_, tempDFs = TimeSeriesDoubleDeltaGrowthAnalysis(df_base, timePeriods_delta = years_delta, predictor='Judicial Effectiveness')

In [5]:
#tempDFs[tempDFs['Index Year'] == 2017]
#tempDFs

### PlotTimeSeriesGrowthAnalysis()
To visualize how changes in the predictor variable (like 'Overall Score') correlate with changes in the target variable (like GDP per capita) for different time shifts.


In [6]:
def PlotTimeSeriesDoubleDeltaGrowthAnalysis(
        df,
        predictor = 'Overall Score',
        target = 'GDP per capita (current USD)',
        quantiledTarget = 'Country Quintile',
        timePeriods_delta = [],
        polyOrder = 1
    ):

    '''
    In a nutshell, a scatterplot of the data created in the function: TimeSeriesDoubleDeltaGrowthAnalysis
    See that function for more information on the variables.
    '''

    # Set up the subplots
    fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 20))
    axes = axes.ravel()
    
    # Loop over each period delta, compute the merged data, and plot the data with the regression line
    for i, delta in enumerate(timePeriods_delta):
        
        predictor_column = f'{predictor}_%diff_from_EMA'
        target_column = f'{target}_change_{delta}'

        # Check if the columns exist in the dataframe
        if predictor_column not in df.columns or target_column not in df.columns:
            print(f"No data for {predictor} with a time delta of {delta}. Skipping plot for this delta.")
            continue
        
        # Extract the features and target variable
        X = df[f'{predictor}_%diff_from_EMA']
        y = df[f'{target}_change_{delta}']
        
        hue = df[quantiledTarget]
        quintile_order = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']

        # Plot the data
        sns.scatterplot(x=X, y=y, hue=hue, ax=axes[i], s=10, palette="RdBu", hue_order=quintile_order)    
                
        # Plot Non-Linear Regression line
        sns.regplot(x=X, y=y, ax=axes[i], order=polyOrder, color='tan', scatter_kws={'s': 0})

        axes[i].set_title(f'Periods Ahead = {delta}')
        axes[i].set_xlabel(f'Recent Change in {predictor}')
        axes[i].set_ylabel(f'% Change in {target}')
        axes[i].legend(title=quantiledTarget)
        
        #if i >= 4:
        #    ymin, ymax, xmin, xmax = -200, 1000, -10, 10
        #else:
        #    ymin, ymax, xmin, xmax = -50, 400, -5, 5
            
        #axes[i].set_ylim(ymin=ymin, ymax=ymax)
        #axes[i].set_xlim(xmin=xmin, xmax=xmax)

    plt.tight_layout()
    plt.show()


## Economic Freedom : Analyzing each aspect on their own
We'll feed our functions with all 12 sub-categories of Heritage's Economic Freedom Index, alongside average aggregations with Overall Score, Rule of Law, Limited Government, Regulatory Efficiency and Open Markets.

In [7]:
df_base['Rule of Law'] = df_base[['Property Rights', 'Government Integrity', 'Judicial Effectiveness']].mean(axis=1)
df_base['Limited Government'] = df_base[['Government Spending', 'Tax Burden', 'Fiscal Health']].mean(axis=1)
df_base['Regulatory Efficiency'] = df_base[['Business Freedom', 'Monetary Freedom', 'Labor Freedom']].mean(axis=1)
df_base['Open Markets'] = df_base[['Financial Freedom', 'Investment Freedom', 'Trade Freedom']].mean(axis=1)

predictors = [
    'Overall Score'
]

In [8]:
predictors = [
    'Overall Score',    
    'Rule of Law',
    'Limited Government',
    'Regulatory Efficiency',
    'Open Markets'
]

In [9]:
# Create an empty DataFrame to store coefficients and R^2 scores for year 6
summaryData = pd.DataFrame(columns=['Predictor', 'Coefficient (Year 6)', 'R^2 Score (Year 6)', 'RMSE (Year 6)'])

polyOrder = 1
EMAdepth = 2

#maskedDF = df_base[df_base['Index Year'] == '2014']

# Loop through each predictor
for predictor in predictors:
    print(f"## {predictor}: Results")
    regression_results, df_withDeltas \
            = TimeSeriesDoubleDeltaGrowthAnalysis(df_base, polyOrder = polyOrder, timePeriods_delta=years_delta, predictor=predictor, EMAdepth=EMAdepth)

    # Extract the coefficient and R^2 score for year 6 
    year6_data = regression_results[regression_results['Years Ahead'] == 6]
    coeff_for_year6 = year6_data['Coefficient (target to predictor % Change)'].values
    r2_for_year6 = year6_data['R^2 Score'].values
    RMSE_for_year6 = year6_data['RMSE'].values

    # Append to the DataFrame
    new_row = pd.DataFrame({
        'Predictor': predictor, 
        'Coefficient (Year 6)': coeff_for_year6,
        'R^2 Score (Year 6)': r2_for_year6,
        'RMSE (Year 6)': RMSE_for_year6
    })
    summaryData = pd.concat([summaryData, new_row], ignore_index=True)

    # Display regression results
    display(regression_results)

    # Plot the analysis
#    PlotTimeSeriesDoubleDeltaGrowthAnalysis(df_withDeltas, 
#                             timePeriods_delta=years_delta, 
#                             predictor=predictor,
#                             polyOrder = polyOrder
#                             )

    print("\n\n")

## Overall Score: Results


C:\Users\gusta\AppData\Local\Temp\ipykernel_1600\4250665895.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  summaryData = pd.concat([summaryData, new_row], ignore_index=True)


,Years Ahead,Coefficient (target to predictor % Change),Intercept,R^2 Score,RMSE
0,1,0.391478,5.355519,0.000380,14.211283
1,2,0.994179,11.487447,0.000954,22.790060
2,3,2.268550,17.948293,0.002525,32.197579
3,4,3.807221,25.928482,0.004012,42.989367
4,5,6.068030,35.219795,0.006177,55.332866
5,6,8.099909,45.535141,0.006353,69.573356
6,8,15.045841,69.595792,0.011082,100.824503
7,10,18.933894,98.941286,0.010336,135.103360
8,12,20.443960,129.859650,0.009511,157.509457
9,15,23.556902,174.104953,0.010034,185.767140





## Rule of Law: Results


,Years Ahead,Coefficient (target to predictor % Change),Intercept,R^2 Score,RMSE
0,1,0.041569,5.344832,0.000014,14.261641
1,2,0.608890,11.500272,0.001203,22.908659
2,3,1.308605,18.017766,0.002777,32.328153
3,4,2.092161,26.130850,0.004044,43.105603
4,5,2.534375,35.546522,0.003563,55.515673
5,6,3.092253,45.780584,0.002783,69.820263
6,8,2.811001,70.271423,0.001076,101.469937
7,10,2.839239,99.886780,0.000667,135.895712
8,12,1.411181,131.035862,0.000131,158.418831
9,15,0.320109,175.995138,0.000006,187.044049


...

,Years Ahead,Coefficient (target to predictor % Change),Intercept,R^2 Score,RMSE
0,1,0.053023,5.360132,0.000045,14.232235
1,2,0.176916,11.476009,0.000187,22.888685
2,3,0.737410,17.945145,0.001574,32.298893
3,4,1.937307,25.875629,0.005523,43.037629
4,5,2.941719,35.206339,0.007489,55.364091
5,6,3.324166,45.576355,0.006012,69.667740
6,8,6.402379,69.631439,0.008590,101.037501
7,10,8.910050,98.903834,0.009491,135.215121
8,12,8.746138,129.444495,0.006221,157.812342
9,15,10.354321,173.536179,0.006828,186.214020





## Regulatory Efficiency: Results


,Years Ahead,Coefficient (target to predictor % Change),Intercept,R^2 Score,RMSE
0,1,0.006325,5.342220,2.999294e-07,14.261741
1,2,0.390581,11.429628,4.519156e-04,22.917270
2,3,0.962511,17.888056,1.415626e-03,32.350207
3,4,2.231997,25.836638,4.384020e-03,43.098248
4,5,4.684556,35.046394,1.204019e-02,55.279013
5,6,7.163870,45.207753,1.831607e-02,69.274340
6,8,13.726596,69.009998,3.282116e-02,99.844603
7,10,18.034058,97.792654,3.308806e-02,133.673151
8,12,24.096836,128.144203,4.785138e-02,154.592205
9,15,23.708815,171.804542,3.857860e-02,183.401123





## Open Markets: Results


,Years Ahead,Coefficient (target to predictor % Change),Intercept,R^2 Score,RMSE
0,1,0.140598,5.324621,0.000135,14.261939
1,2,0.366966,11.404837,0.000362,22.920082
2,3,0.168760,17.957881,0.000038,32.376877
3,4,0.042522,26.065389,0.000001,43.200718
4,5,0.114591,35.551922,0.000006,55.625033
5,6,0.743844,46.093475,0.000173,69.918773
6,8,1.666794,70.882794,0.000441,101.502201
7,10,2.310199,100.600478,0.000508,135.906536
8,12,4.490829,132.027740,0.001552,158.306184
9,15,3.119471,176.264475,0.000574,186.886748


In [10]:
summaryData = summaryData.round(3)
summaryData.sort_values(by='R^2 Score (Year 6)')

,Predictor,Coefficient (Year 6),R^2 Score (Year 6),RMSE (Year 6)
4,Open Markets,0.744,0.000,69.919
1,Rule of Law,3.092,0.003,69.820
0,Overall Score,8.100,0.006,69.573
2,Limited Government,3.324,0.006,69.668
3,Regulatory Efficiency,7.164,0.018,69.274


## Analyzing all features, grouped by each Country Quintile

In [11]:
years_delta = [1, 2, 6, 12, 20]

polyOrder = 1
EMAdepth = 2

def run_regression_for_combination(predictor, delta, year, q, df_withDeltas):
    currentX = f'{predictor}_diff_from_EMA'
    currentY = f'GDP per capita (current USD)_change_{delta}'
    yearMask = df_withDeltas['Index Year'] != year
    quintileMask = df_withDeltas['Country Quintile'] != q
    naMask = yearMask | quintileMask | df_withDeltas[currentX].isna() | df_withDeltas[currentY].isna()
    
    x = df_withDeltas[currentX].values[~naMask].reshape(-1, 1)
    y = df_withDeltas[currentY].values[~naMask]
    
    model = LinearRegression().fit(x, y)
    yHat = model.predict(x)
    
    coeff = model.coef_
    r2 = model.score(x, y)
    RMSE = np.sqrt(mean_squared_error(y, yHat))
    
    return {
        'Predictor': predictor,
        'Index Year': year,
        'Year Delta': delta,
        'Quintile': q,
        'Coefficients': coeff[0],
        'R^2 Scores': r2,
        'RMSEs': RMSE
    }

results = []

for predictor in predictors:
    _ , df_withDeltas = TimeSeriesDoubleDeltaGrowthAnalysis(\
                                                            df_base,\
                                                            timePeriods_delta=years_delta,\
                                                            predictor=predictor)
    #display(df_withDeltas)
    
    for delta in years_delta:
        naMask = df_withDeltas[f'{predictor}_diff_from_EMA'].isna() | df_withDeltas[f'GDP per capita (current USD)_change_{delta}'].isna()        
        years_List = list(df_withDeltas['Index Year'][~naMask].unique())
        for q in ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']:
            for year in years_List:
                result = run_regression_for_combination(predictor, delta, year, q, df_withDeltas)
                results.append(result)

summaryData_agg = pd.DataFrame(results)

In [12]:
summaryData_agg.drop(columns=['Quintile', 'Index Year']).groupby(['Predictor', 'Year Delta'])\
    .mean().sort_values(by=['Predictor', 'Year Delta']).round(3)



Coefficients  R^2 Scores    RMSEs
Predictor             Year Delta                                   
Limited Government    1             -14879.683       0.081   10.194
                      2             -10448.984       0.048   16.543
                      6             -32807.823       0.046   43.825
                      12            251528.730       0.038  103.266
                      20           1017845.523       0.041  163.804
Open Markets          1             -84401.746       0.074   10.243
                      2            -105884.081       0.034   16.746
                      6             -24406.375       0.037   44.148
                      12           -273590.935       0.031  104.073
                      20           1058119.192       0.039  165.527
Overall Score         1              43113.442       0.087   10.100
                      2              51915.800       0.044   16.467
                      6              61322.323       0.044   43.491
                      12            354084.866       0.039  102.677
                      20          -1239203.695       0.057  162.416
Regulatory Efficiency 1              38235.671       0.079   10.280
                      2              80835.627       0.046   16.645
                      6             150172.417       0.043   43.614
                      12           -706314.925       0.048  102.476
                      20           -745129.516       0.055  163.778
Rule of Law           1              -4918.958       0.075   10.239
                      2             -48094.693       0.032   16.752
                      6             -24731.966       0.036   44.028
                      12          -2100032.746       0.039  103.472
                      20          -5769804.780       0.045  165.077

In [13]:
summaryData_agg.drop(columns=['Quintile', 'Index Year', 'Predictor']).describe().round(3)

,Year Delta,Coefficients,R^2 Scores,RMSEs
count,2475.000,2.475000e+03,2475.000,2475.000
mean,5.687,-1.688992e+05,0.052,46.830
std,5.709,6.361210e+06,0.112,63.352
min,1.000,-1.316420e+08,0.000,0.000
25%,1.000,-1.425000e+00,0.005,11.247
50%,2.000,3.680000e-01,0.020,21.455
75%,6.000,2.615000e+00,0.057,48.283
max,20.000,9.343915e+07,1.000,433.223
